In [8]:
# 환경변수로 설정된 OpenAI API Key 가져오기
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

### Dynamic few-shot prompting
더 좋은 결과를 자동으로 가져오기 위해..

In [9]:
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts.example_selector.semantic_similarity import SemanticSimilarityExampleSelector
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models.openai import ChatOpenAI
import sqlite3

print(sqlite3.version)

examples = [
{"input": "2+2", "output": "4"},
{"input": "2+3", "output": "5"},
{"input": "2+4", "output": "6"},
{"input": "What did the cow say to the moon?", "output": "nothing at all"},
{
"input": "Write me a poem about the moon",
"output": "One for the moon, and one for me, who are we to talk about the moon?",
},
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
# The input variables select the values to pass to the example_selector
input_variables=["input"],
example_selector=example_selector,
# Define how each example will be formatted.
# In this case, each example will become 2 messages:
# 1 human, and 1 AI
example_prompt=ChatPromptTemplate.from_messages(
[("human", "{input}"), ("ai", "{output}")]
),
)
final_prompt = ChatPromptTemplate.from_messages(
[
("system", "You are a wondrous wizard of math."),
few_shot_prompt,
("human", "{input}"),
]
)
chain = final_prompt | ChatOpenAI(temperature=0.0)
chain.invoke({"input": "What's 3+3?"})


2.6.0


RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0.[0m
[94mPlease visit https://docs.trychroma.com/troubleshooting#sqlite to learn how to upgrade.[0m

### PromptTemplate

In [ ]:
#0) PromptTemplate.from_template() or PromptTemplate constructor
prompt_template = PromptTemplate.from_template("Tell me a {adjective} story about {content}.")
prompt_template = PromptTemplate( input_variables = ['adjective', 'content'], template = 'Tell me a {adjective} story about {content}')

#1) format to StringPromptValue
value = prompt_template.format_prompt(adjective="funny", content="glasses")

#2) PromptValue.to_string(), PromptValue().to_messages()
value.to_string() == string
value.to_messages()
[HumanMessage(content='Tell me a funny story about glasses.', additional_kwargs={}, example=False)]

#3) format to string
string = prompt_template.format(adjective="funny", content="glasses“)


### ChatPromptTemplate
PromptTemplate와 비교

In [ ]:
# 0) ChatPromptTemplate.from_message() or ChatPromptTemplate - constructor
chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful AI assistant. Your name is {name}"),
    ('human', "Hello, how are you doing?"),
    ('ai', "I'm doing well, thanks!"),
    ('human', "{user_input}")
])

# 1) format to ChatPromptValue
value = chat_prompt_template.format_prompt(name='Bob', user_input="What is your name?")

# 2) ChatPromptValue.to_string(), ChatPromptValue().to_messages()
value.to_string() == string
value.to_messages() == msg

# 3) format to string
string = chat_prompt_template.format(name='Bob', user_input="What is your name")

# 4) format to messagges - 추가된거
msg = chat_prompt_template.format_messages(name='Bob', user_input="What is your name?")


### Example Samplers
* 몇 가지 예제를 선택함으로써 LLM 모델이 유저의 요청을 더 잘 이해
* 유저의 질문과 관련된 예제를 잘 설계하면 LLM 성능 향상
* LangChain은 5가지 예제 샘플러 제공

In [ ]:
# 1. Select by Length : LengthBasedExampleSelector
# 2. Select by Maximal Margin Relevance : MaxMarginalRelevanceExampleSelector
# 3. Select by n-gram similarity : NGramOverlapExampleSelector :
# 4. Select by Similarity : SemanticSimilarityExampleSelector
# 5. Custom Selector : CustomExampleSelector

### Overall Steps for Dynamic Prompting
내 정확한 의도를 알려주기 위함?

다이나믹 프롬프팅의 전반적인 절차
* 중요

1. 내 앱의 예제를 준비 (데이터셋을 잘 만드는게 중요)
2. 프롬프트 템플릿 선언
3. 적절한 Example Sampler를 선택
4. few shot prompt template에 예제, 프롬프트 템플릿, 예제 샘플러 합침
5.
6.

1) Select by Similarity

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples, 
    # The embedding class used to produce embeddings 
    # which are used to measure semantic similarity.
    OpenAIEmbeddings(), 
    # The VectorStore3class that is used to store the embeddings 
    # and do a similarity search over.
    Chroma, # FAISS
    # The number of examples to produce.
    k=1
)

"\s+" means one or more whitespace characters

5) Custom Sampler – random example

### Dynamic Prompting

In [11]:
# Step 1: Prepare Examples

# Examples of a pretend task of creating antonyms.
examples_1 = [
{"input": "happy", "task":"in antonym", "output": "sad"},
{"input": "tall", "task":"in antonym", "output": "short"},
{"input": "energetic", "task":"in antonym", "output": "lethargic"},
{"input": "sunny", "task":"in antonym", "output": "gloomy"},
{"input": "windy", "task":"in antonym", "output": "calm"},
]
# Examples of a fictional translation task.
examples_2 = [
{"input": "See Spot run.", "task": "in spanish", "output": "Ver correr a Spot."},
{"input": "My dog barks.", "task": "in spanish", "output": "Mi perro ladra."},
{"input": "Spot can run.", "task": "in spanish", "output": "Spot puede correr."},
]

# Step 2: Define Prompt Template
# 2. We define prompt for examples.
from langchain.prompts import PromptTemplate
example_prompt = PromptTemplate(
    input_variables=['input', 'task', 'output'],
    template="Input: {input} --> {task}\nOutput: {output}"
)

# Step 3: Define Example Selector
# 3.1 we define semantic-similarity-example-selector
# 3.2 we define maximal-relevance-example-sampler
# 3.3 we define n-gram-overlap-example-sampler
# 3.4 we define length-based-example-sampler
# 3.5 we define custom-example-sampler
# 위에서 본 5가지 중 한가지 선택하면 됨

sms_sampler = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    OpenAIEmbeddings(), 
    Chroma, # FAISS
    k=1
)


# Step 4: Assemble examples, prompt template, and example sampler
# 4. We assemble sampler and prompt into few-shot-prompt-template for dynamic prompting
from langchain.prompts import FewShotPromptTemplate
def dynamic_prompt(sampler, example_prompt):
    return FewShotPromptTemplate(
        example_selector=sampler,
        example_prompt=example_prompt,
        prefix="Here are some examples related to the query!\n"
        ,
        suffix=
        "{query}:\nOutput: ",
        input_variables=["query"]
    )
    
# Step.5 Generate A Dynamic Prompt
# 5. Generate a prompt associated with an input query.
user_query = "windy in spanish"
# sms_sampler: 위에서 본 샘플러 중 하나 (아마 시맨틱 시뮬러리티 샘플러 쓴 듯)
prompt = dynamic_prompt(sms_sampler, example_prompt).format(query=user_query)
print(prompt)

'''
Here are some examples related to the query!
Input: windy --> in antonym
Output: calm
Input: My dog barks. --> in spanish
Output: Mi perro ladra.
windy in spanish:
Output:
'''


RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0.[0m
[94mPlease visit https://docs.trychroma.com/troubleshooting#sqlite to learn how to upgrade.[0m

### Composition – Pipeline Prompt
프롬프트 여러개 섞기
full_prompt = PromptTemplate.from_template("""{introduction}

{example}
{start}""")

여기 {} 섹션 안에 채워 넣는 듯

Output Parser
프롬프팅 할 때, 어떤 출력으로 할 지도 프롬프팅 가능함
가령, 딕셔너리나 리스트 형태로 출력해달라는지
특정 형태로 출력해달라고 하고, 답이 오면 파싱해서 파이썬에서 사용 가능한 형태로?

pydantic output parser example

출력을 파이썬 클래스로 파싱해가지고 출력 받을 수도 있음
내가 원하는 포멧으로 받을 수 있다.